# Connect to the cluster

In [ ]:
import socket
if socket.gethostname() == 'io':
    import hpc05
    client = hpc05.connect.start_remote_and_connect(200, folder='~/Work/two_dim_majoranas')[0]
else:
    # No not connect to the cluster in CI for example.
    client = None

## Numerics imports

In [ ]:
from functools import partial
from copy import copy
import numpy as np
import adaptive
adaptive.notebook_extension()

## Plotting imports

In [ ]:
import matplotlib
matplotlib.use('agg')

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

golden_mean = (np.sqrt(5) - 1) / 2 # Aesthetic ratio
fig_width_pt = 246.0 # Columnwidth
inches_per_pt = 1.0 / 72.27 # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean # height in inches
fig_size = [fig_width, fig_height]

params = {
          'backend': 'ps',
          'axes.labelsize': 13,
          'font.size': 13,
          'legend.fontsize': 10,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 300,
         }

plt.rcParams.update(params)
plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}', r'\usepackage{xfrac}'])

## Parameter and system definitions

In [ ]:
import scipy.constants
import cmath

constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle=26, g_factor_left=0, g_factor_right=0,
                 mu=10,
                 alpha_middle=20, alpha_left=0, alpha_right=0,
                 Delta_left=1, Delta_right=1,
                 B_x=0,
                 B_y=0,
                 B_z=0,
                 phase=0,
                 T=0.0,
                 V=0)

params = dict(**constants,
              **params_raw)

syst_pars = dict(
    L_m=200,
    L_x=1300,
    L_sc_up=200,
    L_sc_down=200,
    z_x=1300,
    z_y=0,
    a=10,
    parallel_curve=True,
    sawtooth=False,
    transverse_soi=True,
    mu_from_bottom_of_spin_orbit_bands=True,
    k_x_in_sc=True,
    wraparound=True,
    current=False,
    ns_junction=False)

# Figure 2. Bandstructures

In [ ]:
def spectrum_wrapper(k_x, z_y, B_x, phase, syst_pars=copy(syst_pars), params=copy(params), nbands=40):
    import numpy as np
    import sns_system, spectrum
    params = dict(params, k_x=k_x, B_x=B_x, phase=phase)
    syst_pars = dict(syst_pars, z_y=z_y)
    syst_at_k, _, _ = sns_system.make_system(**syst_pars)
    energies = spectrum.calc_spectrum(syst_at_k, params, k=nbands)[0]
    return np.sort(energies)

def abs_min_log_loss(xs, xy_scale, data):
    from adaptive.learner.learner1D import default_loss
    data = {k: np.log(np.abs(v).min()) for k, v in data.items()}
    return default_loss(xs, xy_scale, data)

B = 1
W = syst_pars['L_m']
combos = [(0, 0, 0), (W/4, 0, 0), (W/2, 0, 0), (0, B, np.pi), (W/4, B, np.pi), (W/2, B, np.pi)]

learners = [adaptive.Learner1D(partial(spectrum_wrapper, z_y=z_y, B_x=B_x, phase=phase),
                               bounds=[-np.pi, np.pi], loss_per_interval=abs_min_log_loss)
            for z_y, B_x, phase in combos]

learner = adaptive.BalancingLearner(learners, cdims=(['z_y', 'B_x', 'phi'], combos), strategy='npoints')

folder = 'data/bandstructures'
for l, combo in zip(learners, combos):
    l.fname = f'spectrum_{combo}.pickle'
learner.load(folder)

runner = adaptive.Runner(learner, lambda l: l.learners[0].npoints > 200, executor=client)
runner.live_info()

In [ ]:
mapping = {(z_y, B_x, phase): learner for (z_y, B_x, phase), learner in zip(combos, learners)}

def plot(ax, key, color):
    data = mapping[key].data
    xs, ys = map(np.array, zip(*sorted(data.items())))
    return ax.plot(xs, ys, c=color)

fig, axs = plt.subplots(3, sharex=True, sharey=True, figsize=(fig_width, 2*fig_height))
(ax1, ax2, ax3) = axs

line2 = plot(ax1, (0, B, np.pi), 'red')[0]
line1 = plot(ax1, (0, 0, 0), 'blue')[0]

plot(ax2, (W/4, B, np.pi), 'red')
plot(ax2, (W/4, 0, 0), 'blue')

plot(ax3, (W/2, B, np.pi), 'red')
plot(ax3, (W/2, 0, 0), 'blue')

for i, ax in enumerate(axs):
    ax.set_ylabel(r'$E/\Delta$')
    
    # ylims and yticks
    ax.set_ylim(-0.35, 0.35)
    yvals = [-0.3, 0, 0.3]
    ylabels = [f'${x}$' for x in yvals]
    ax.set_yticks(yvals)
    ax.set_yticklabels(ylabels)
    
    # xlims and xticks
    ax.set_xlim(-3, 3)
    xvals = [-np.pi, -np.pi/2, 0, np.pi/2, np.pi]
    xlabels = ['$\sfrac{-\pi}{a}$', '$\sfrac{-\pi}{2a}$', r'$0$', '$\sfrac{\pi}{2a}$', r'$\sfrac{\pi}{a}$']
    ax.set_xticks(xvals)
    ax.set_xticklabels(xlabels)

    # text inside image
    label = 'abc'[i]
    ax.text(0.95, 0.5, f'$\mathrm{{({label})}}$', transform=ax.transAxes,
            verticalalignment='center', horizontalalignment='center')
    
    z_ys = ['0', 'W/4', 'W/2']
    ax.text(0.01, 0.5, f'$z_y={z_ys[i]}$', transform=ax.transAxes,
            verticalalignment='center', horizontalalignment='left')

ax1.legend((line1, line2), (r'$\phi=0$, $B=0$', r'$\phi=\pi$, $B \ne 0$'),
           loc='upper center', bbox_to_anchor=(0.5, 1.7),
           fancybox=True, shadow=False, ncol=1)

ax3.set_xlabel('$k_x$')

plt.savefig("paper/figures/bandstructures.pdf", bbox_inches="tight")
plt.show()

# Figure 3. Phase diagrams

TODO: add the phase boundaries

In [ ]:
syst_pars['L_x'] = 10  # XXX: tmp setting this to a low number for speed

def gap_wrapper(xy, keys, z_y, syst_pars=copy(syst_pars), params=copy(params), nbands=40):
    import numpy as np
    import sns_system, spectrum
    from scipy.optimize import brute
    syst_pars = dict(syst_pars, z_y=z_y)
    syst_at_k, _, _ = sns_system.make_system(**syst_pars)
    for k, _xy in zip(keys, xy):
        params[k] = _xy

    def energies(k_x):
        return np.abs(spectrum.calc_spectrum(syst_at_k,
            dict(params, k_x=float(k_x)), k=nbands)[0]).min()

    return brute(energies, ranges=((0, np.pi),), Ns=201)

B = 1
W = syst_pars['L_m']
combos = [(0, 'mu', (0, 10)), (0, 'phi', (0, 2*np.pi)),
          (W/2, 'mu', (0, 10)), (W/2, 'phi', (0, 2*np.pi))]

learners = [adaptive.Learner2D(partial(gap_wrapper, keys=[key, 'B_x'], z_y=z_y),
                               bounds=[xbounds, (0, 2)])
            for z_y, key, xbounds in combos]

learner = adaptive.BalancingLearner(learners, cdims=(['z_y'], combos), strategy='npoints')

folder = 'data/phase_diagrams'
for l, combo in zip(learners, combos):
    l.fname = f'phase_diagram_{combo}.pickle'
learner.load(folder)

runner = adaptive.Runner(learner, lambda l: l.learners[0].npoints > 5000, executor=client)
runner.live_info()

In [ ]:
mapping = {(z_y, key): learner for (z_y, key, xbounds), learner in zip(combos, learners)}

def plot(ax, z_y, key):
    learner = mapping[(z_y, key)]
    im = learner.plot().Image.I
    l, b, r, t = im.bounds.lbrt()
    return ax.imshow(im.data, extent=(l, r, b, t), aspect='auto')

fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(fig_width, 2*fig_height))
(ax1, ax2), (ax3, ax4) = axs

for ax, (z_y, key, xbounds) in zip(axs.reshape(-1), combos):
    plot(ax, z_y, key)

for i, ax in enumerate(axs.reshape(-1)):
    if i in [0, 2]:
        ax.set_ylabel(r'$B_x$ (T)')

    # ylims and yticks
    yvals = [0, 1, 2]
    ylabels = [f'${x}$' for x in yvals]
    ax.set_yticks(yvals)
    ax.set_yticklabels(ylabels)
    
    # xlims and xticks
    if i in [1, 3]:
        xvals = [0, np.pi, 2*np.pi]
        xlabels = [r'$0$', '$\pi$', r'$2\pi$']
        ax.set_xticks(xvals)
        ax.set_xticklabels(xlabels)

    # text inside image
    label = 'abcd'[i]
    ax.text(0.99, 0.98, f'$\mathrm{{({label})}}$', transform=ax.transAxes,
            verticalalignment='top', horizontalalignment='right',
            color='white')
    
    z_y = ['0', '0', 'W/4', 'W/4'][i]
    ax.text(0.99, 0.01, f'$z_y={z_y}$', transform=ax.transAxes,
            verticalalignment='bottom', horizontalalignment='right',
            color='white')

ax3.set_xlabel('$\mu$ (meV)')
ax4.set_xlabel('$\phi$ (rad)')

plt.savefig("paper/figures/phasediagrams.pdf", bbox_inches="tight")
plt.show()

# Figure 4. Wavefunctions